<a href="https://colab.research.google.com/github/flydevilz/automate_zoom_attendance/blob/main/Check_Presensi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Use:
1. Download CSV dari ZOOM. Setelah itu masukkan csv di GDrive masing2.
2. Set share -> general access -> anyone with the link -> copy link
3. Runtime -> Run All
4. Paste link di bagian insert link, kemudian enter.
5. Tunggu semuanya selesai +- 5 detik, dan silahkan lihat attendance.



In [1]:
insert_link = str(input())

https://drive.google.com/file/d/1N8J3y61SrsCQoyZu9pSwaxNYIEG4HTrj/view?usp=drive_link


# Import Data

In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
pattern_link = r"https://drive\.google\.com/file/d/([\w-]+)/view\?usp=drive_link"
replacement = r"https://drive.google.com/uc?id=\1"

In [4]:
# ubah link level3 jd link spreadsheet masing2
link_airtable= 'https://drive.google.com/uc?id=1gIKKnVxugWQd_WxuOyGNSQRuPli_AOfw'
link_attendance = re.sub(pattern_link, replacement, insert_link)
link_level3= 'https://drive.google.com/uc?id=1JrZp_Ve6mekR-dPqR8xQBAlrgZtBjXqw'

In [55]:
airtable = pd.read_csv(link_airtable)
attendance = pd.read_csv(link_attendance, skiprows=2)
level3 = pd.read_csv(link_level3, skiprows=2)

# Clean Airtable

In [56]:
# ubah nama section
airtable_semarang = airtable[(airtable['Section'] == 'Semarang')]

In [57]:
# filter kolom yg dibutuhkan
airtable_name_email = airtable_semarang[['Student ID', 'Student Name', 'Email', 'Email RevoU']]

In [58]:
# rename column nama supaya jelas
airtable_name_email = airtable_name_email.rename(columns= {'Student ID':'id','Student Name': 'name', 'Email RevoU':'email_revou'})

In [59]:
# clean double space di nama
airtable_name_email['name'] = airtable_name_email['name'].str.strip().str.replace(r'\s+', ' ')
# buat all name jadi uppercase
airtable_name_email['name'] = airtable_name_email['name'].str.upper()

In [60]:
# ubah data type Student ID supaya gampang di merge
airtable_name_email['id'] = airtable_name_email['id'].astype('int64')

# Clean Attendance Zoom

In [61]:
# rename column nama supaya jelas
attendance_name_email = attendance.rename(columns= {'Name (Original Name)':'name','User Email': 'email', 'Duration (Minutes)':'duration'})

In [62]:
# filter kolom yg dibutuhkan
attendance_name_email = attendance_name_email[['name', 'email', 'duration']]

In [63]:
# clean double space di nama
attendance_name_email['name'] = attendance_name_email['name'].str.strip().str.replace(r'\s+', ' ')
# buat all name jadi uppercase
attendance_name_email['name'] = attendance_name_email['name'].str.upper()

# Clean Level 3

In [64]:
level3.dropna(subset=['Student ID'], inplace=True)

In [65]:
# ubah data type supaya gampang di merge
level3['Student ID'] = level3['Student ID'].astype('int64')
level3['Team'] = level3['Team'].astype('int64')

In [66]:
# rename column nama supaya jelas
level3 = level3.rename(columns= {'Student ID':'id','Student Name': 'name', 'Email':'email'})

In [67]:
# clean double space di nama
level3['name'] = level3['name'].str.strip().str.replace(r'\s+', ' ')
# buat all name jadi uppercase
level3['name'] = level3['name'].str.upper()

In [68]:
# filter kolom yg dibutuhkan
absensi = level3.iloc[:, [0, 1, 2, 3, -1]]

In [69]:
absensi = absensi[['Team','name','email']]

## Check email kalau udah sama dengan airtable

In [ ]:
dum_lv3= level3[['id','email']]

In [ ]:
dum_email_air = airtable_name_email[['id','name', 'email_revou']]

In [ ]:
check = pd.merge(dum_email_air, dum_lv3, on='id', how='left', suffixes = ('_airtable', '_level3'))

In [ ]:
check['Notes'] = check['email_revou'] == check['email']

In [ ]:
mismatched_email = check[check['Notes'] == False]

In [ ]:
mismatched_email

Kalau sama berarti aman :) Lanjut step berikut

# Next Step

In [70]:
attendance_name_email.shape

(556, 3)

In [71]:
# mengisi email kosong dengan "-"
attendance_name_email['email'].fillna('-', inplace=True)

In [72]:
# calculate the sum of the duration
total_duration = attendance_name_email.groupby(['name', 'email'], as_index=False)['duration'].sum()

# Inner Join

In [73]:
# inner join by email
same = pd.merge(absensi, total_duration, how='inner', on = 'email', suffixes= ('_level3', '_zoom'))

In [74]:
# drop irrelevant column
same.drop(columns=['name_zoom'], inplace = True)

In [80]:
# group by
z = same.groupby(['email'])['duration'].sum().reset_index()

In [81]:
p = pd.merge(absensi, z, how='left', on = 'email')

# Outer Join

In [85]:
# merge outer attendance dengan level 3
att_check = pd.merge(absensi, total_duration, how='outer', on = 'email', suffixes= ('_level3', '_zoom'))

In [86]:
condition_1 = (att_check['name_level3'].isnull()) & (att_check['name_zoom'].notnull())
hadir_zoom_ga_kedetect = att_check[condition_1]
hadir_zoom_ga_kedetect = hadir_zoom_ga_kedetect[['name_zoom', 'duration']]

In [87]:
condition_2 = (att_check['name_level3'].notnull()) & (att_check['name_zoom'].isnull())
yg_mau_dimerge = att_check[condition_2]
yg_mau_dimerge = yg_mau_dimerge[['Team','name_level3','email']]

In [30]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Yang join ZOOM

In [88]:
right = pd.merge(yg_mau_dimerge, hadir_zoom_ga_kedetect, left_on = 'name_level3', right_on= 'name_zoom', how='right')

In [89]:
condition_3 = (right['name_level3'].notnull()) & (right['email'].notnull()) & (right['name_zoom'].notnull())
fix = right[condition_3]

### TAMBAH PRESENSI (YG GA SESUAI EMAIL TAPI SAMA NAMA) AKA MATCH BY NAME YG DURUASINYA >=60

In [90]:
sorted_fix = fix.sort_values(by='duration')
hadir_diatas_60 = sorted_fix[sorted_fix['duration'] >= 60]

In [91]:
hadir_diatas_60.sort_values(by='Team', ascending=True)

,Team,name_level3,email,name_zoom,duration
11,6.0,MUHAMMAD RAMDANI YUSUF,ramdaniyusuf69@gmail.com,MUHAMMAD RAMDANI YUSUF,82.0
13,14.0,NURUL ROFIDHATUL ILMI,nurul.rofidhatul112@gmail.com,NURUL ROFIDHATUL ILMI,127.0
3,14.0,AMINATUS ZAHRO,zahroamina93@gmail.com,AMINATUS ZAHRO,128.0
5,22.0,FARRELL ABRAR RHIZNANDA,farrellabrar18@gmail.com,FARRELL ABRAR RHIZNANDA,151.0


# Download

In [94]:
# buang kolom yg ga perlu
q = hadir_diatas_60.drop(columns=['name_zoom'])

In [99]:
# rename nama
q.rename(columns={'name_level3': 'name'}, inplace=True)

In [107]:
# merge antara data inner join yg match by email dan data yg match by name
z= pd.merge(p, q, how='left', on = 'email')

In [109]:
# isi durasi
z['duration_x'] = z['duration_y'].fillna(z['duration_x'])

In [113]:
# drop column
z= z.drop(columns=['Team_y', 'name_y', 'duration_y'])

# rename column
z= z.rename(columns={'Team_x':'team', 'name_x': 'name', 'duration_x': 'duration'})

In [122]:
# Add note

conditions = [
    (z['duration'] >= 60),
    (z['duration'] < 60)
]

# Assign values based on conditions
values = ['P', '0']

# Use numpy's select function to assign values based on conditions
z['note'] = np.select(conditions, values, default=np.nan)

# Replace string 'nan' becomes NaN
z['note'] = z['note'].replace('nan', np.nan)

In [132]:
z.to_excel('presensi.xlsx', index=False)

In [133]:
try:
    from google.colab import files
    files.download('presensi.xlsx')
except ImportError:
    pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Yang Ga Ke Detect

In [ ]:
# @title
outer = pd.merge(yg_mau_dimerge, hadir_zoom_ga_kedetect, left_on = 'name_level3', right_on= 'name_zoom', how='outer')

In [ ]:
# @title
condition_4 = (outer['name_level3'].notnull()) & (outer['email'].notnull())
f = outer[~(condition_4)]

In [ ]:
f.shape

(15, 5)

In [ ]:
f

,Team,name_level3,email,name_zoom,duration
48,NaN,NaN,NaN,2502038311 - SOFIA SHANTY SUGIYONO,13.0
49,NaN,NaN,NaN,ABDULLAH HENDRIAWAN SECTION SEMARANG TEAM 8 (S...,117.0
50,NaN,NaN,NaN,AKMAL,118.0
51,NaN,NaN,NaN,ARGA ADYATAMA,148.0
52,NaN,NaN,NaN,H1401211035_IHSAN HIDAYAT,127.0
53,NaN,NaN,NaN,HIZBULLAH (PCHOLDER),133.0
54,NaN,NaN,NaN,MALVIN - MENTOR,126.0
55,NaN,NaN,NaN,MOCH ARDHI KURNIAWAN,142.0
56,NaN,NaN,NaN,MUHAMMAD RIVALDI PRABOWO,101.0
57,NaN,NaN,NaN,RAIHAN BAIHAQI,1.0


In [ ]:
pattern = r'^\d+_([A-Z\s]+)'
f['name_zoom'] = f['name_zoom'].str.replace(pattern, r'\1', regex=True)
pattern2 = r'^\d+\s+([A-Z\s]+)'
f['name_zoom'] = f['name_zoom'].str.replace(pattern2, r'\1', regex=True)

<ipython-input-112-5ffb9168a6e7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f['name_zoom'] = f['name_zoom'].str.replace(pattern, r'\1', regex=True)
<ipython-input-112-5ffb9168a6e7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f['name_zoom'] = f['name_zoom'].str.replace(pattern2, r'\1', regex=True)


In [ ]:
total_duration1 = f.groupby(['name_zoom'], as_index=False)['duration'].sum()

In [ ]:
f.dropna(subset=['name_zoom', 'duration'], inplace=True)

<ipython-input-114-a0f4cd478d51>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f.dropna(subset=['name_zoom', 'duration'], inplace=True)


In [ ]:
pattern3 = r'\((.*?)\)'
f['name_zoom'] = f['name_zoom'].str.replace(pattern3, '', regex=True).str.strip()

<ipython-input-115-40ad52d32664>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f['name_zoom'] = f['name_zoom'].str.replace(pattern3, '', regex=True).str.strip()


In [ ]:
total_duration1 = f.groupby(['name_zoom'], as_index=False)['duration'].sum()

In [ ]:
total_duration1

,name_zoom,duration
0,2502038311 - SOFIA SHANTY SUGIYONO,13.0
1,ABDULLAH HENDRIAWAN SECTION SEMARANG TEAM 8,117.0
2,AGIT,152.0
3,AKMAL,118.0
4,ARGA ADYATAMA,148.0
5,H1401211035_IHSAN HIDAYAT,127.0
6,HIZBULLAH,133.0
7,MALVIN - MENTOR,126.0
8,MOCH ARDHI KURNIAWAN,142.0
9,MUHAMMAD RIVALDI PRABOWO,101.0


In [ ]:
f

,Team,name_level3,email,name_zoom,duration
48,NaN,NaN,NaN,2502038311 - SOFIA SHANTY SUGIYONO,13.0
49,NaN,NaN,NaN,ABDULLAH HENDRIAWAN SECTION SEMARANG TEAM 8,117.0
50,NaN,NaN,NaN,AKMAL,118.0
51,NaN,NaN,NaN,ARGA ADYATAMA,148.0
52,NaN,NaN,NaN,H1401211035_IHSAN HIDAYAT,127.0
53,NaN,NaN,NaN,HIZBULLAH,133.0
54,NaN,NaN,NaN,MALVIN - MENTOR,126.0
55,NaN,NaN,NaN,MOCH ARDHI KURNIAWAN,142.0
56,NaN,NaN,NaN,MUHAMMAD RIVALDI PRABOWO,101.0
57,NaN,NaN,NaN,RAIHAN BAIHAQI,1.0


In [ ]:
# run this to check if there's some name you want to check
def check_name(name):
  contains_name = (f['name_zoom'].str.contains(name, na=False)) | (filtered_fix['name_zoom'].str.contains(name, na=False)) | (hadir_diatas_60['name_zoom'].str.contains(name, na=False))
  return f[contains_name]

In [ ]:
# and this, just change the name
result = check_name('IHSAN')
result

<ipython-input-119-37d14bf88415>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  return f[contains_name]


,Team,name_level3,email,name_zoom,duration
52,NaN,NaN,NaN,H1401211035_IHSAN HIDAYAT,127.0
